## Import tools

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import time, random
import pandas as pd

## Check if scraping allowed with robots.txt

In [2]:
url_robots = 'https://www.tapwage.com/'
response_robots = requests.get(url_robots)
print(response_robots.text)

<!DOCTYPE html>
<html lang="en-US">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="referrer" content="always">
    <meta name="csrf-param" content="_csrf">
    <meta name="csrf-token" content="WERxTgY0xwJXM7GT0hdV-HJAQTfUMPuzJGnRc8f-qakpPUkUZxm9NQJY3Prnf2WvEAo7c4tjqv11J4IJnrX-7w==">
    <title>Job Search and Career Advice | Tapwage Job Search</title>
    <link rel="apple-touch-icon" sizes="180x180" href="https://static.tapwage.com/favicons/apple-touch-icon.png">
    <link rel="icon" type="image/png" href="https://static.tapwage.com/favicons/favicon-32x32.png" sizes="32x32">
    <link rel="icon" type="image/png" href="https://static.tapwage.com/favicons/favicon-16x16.png" sizes="16x16">
    <link rel="manifest" href="https://static.tapwage.com/favicons/manifest.json">
    <link rel="mask-icon" href="https://static.tapwage.com/favicons/safari-pinned-tab.svg" color="#5bbad5 ">
    <meta name="theme-color" content=

In [11]:
## Try to scrape a single JD
url = 'https://tapwage.com/job/49810371-phd-scientist-internship?context=8mTP4g47UKLKNXZRpi7RGR3QBt1KRtsuNHkHYmHkencdkwLFVTQyerAYtMDz8U4a5XCkZXFwdiueUaB1ZhGNM7JdiMwsrB7hhCuGPwdkNTKZWn3Hd1n7tVAx99MkGSfcGMcTAerjypqivcbixWgYuNUMQXUu2sZqhPggLzc6'
#url = 'https://tapwage.com/job/48304470-phd-program-administrator-academic-services-senior-professional?context=8mTP4g47UKLKNXZRpi7RGR3QBt1KRtsuNHkHYmHkencdkwLFVTQyerAYtMDz8U4a5XCkZXFwdiueUaB1ZhGNM7JdiMwsrB7hhCuGPwdkNTKZWn3Hd1n7tVAx99MkGSfcGMcTAerjypqivcbixWgYuNUMQXUu2sZqhPggLzc6'
response = requests.get(url)
response.status_code

200

In [12]:
## put into BeautifulSoup
page_html = response.text
soup = bs(page_html)
type(soup)

bs4.BeautifulSoup

### Define function to extract one Job Description

In [203]:
def get_one_jd(soup):

## place to store each JD
    data = {}
## Scrape title box, split into four different columns containing the following info:
## Title, Company, Location, Unique ID
    title = soup.find('title').text
    split1 = title.split('|')
        
    try:
        split2= split1[0].split(' at ')
        data['title'] = split2[0]
    except IndexError:
        print('IndexError: split2')
        print(title)
        data['title'] = title
    
    try:
        split3 = split2[1].split(' in ')
        data['company'] = split3[0]
    except IndexError:
        print('IndexError: split3')
        print(title)
        data['company'] = title
        
    try:
        split4 = split3[1].split(' - ')
        data['location'] = split4[0]
        data['uid'] = split4[1]
    except IndexError:
        print('IndexError: split4')
        print(title)
        print
        data['location'] = title
        data['uid'] = title
        

## Scrape JD tags and put in column
    tags = soup.find(class_ = 'tags')
    tag_list = []
    for item in tags.find_all('a'):
        tag_list.append(str.strip(item.text))
    data['tags'] = tag_list

## Scrape body of JD and put in a single column
    paragraphs = soup.find(class_ = 'description-preview')
    cleaned = paragraphs.text.replace('\xa0', ' ')
    data['description'] = str.strip(cleaned)

## Scrape bullet points in JD and put in a single column
    bullets = []
    for bullet in paragraphs.find_all('li'):
        bullets.append(str.strip(bullet.text))
    data['bullets'] = bullets
    
## Scrape headers and put in a single column
    paragraphs = soup.find(class_ = 'description-preview')
    headers = []
    for item in paragraphs.find_all('strong'):
        headers.append(str.strip(item.text))
    for item in paragraphs.find_all('h2'):
        headers.append(str.strip(item.text))
    data['headers'] = headers

    return(data)




### Define function to extract multiple Job Descriptions from a list of urls to individual Job Description pages

In [66]:
def get_multiple_jds(url_list):
    all_data = []
    for item in url_list:
        time.sleep(2)
        response = requests.get(item)
        page_html = response.text
        soup = bs(page_html)
        all_data.append(get_one_jd(soup))
    return all_data

In [67]:
## See if it works on three test urls
test_list = ['https://tapwage.com/job/48022361-phd-program-lab-manager-hybrid?context=8mTVBWnrY7qYbw3jzBR8gRRa3FV6D7M5a9JBxNb2A7Jcn83xJn6yqVk9dFc7mNJRCRSmhv4AAZxtNuRMaVhNiKbuR2f7nKN4fYenp2zM8FX5eRcDgwfVkWRxVa3GCh9jtaqQ3g7cLuKSKTcCW6TfnJVEiSEEJLVFFDeuGPUK',
             'https://tapwage.com/job/48304470-phd-program-administrator-academic-services-senior-professional?context=8mTP4g47UKLKNXZRpi7RGR3QBt1KRtsuNHkHYmHkencdkwLFVTQyerAYtMDz8U4a5XCkZXFwdiueUaB1ZhGNM7JdiMwsrB7hhCuGPwdkNTKZWn3Hd1n7tVAx99MkGSfcGMcTAerjypqivcbixWgYuNUMQXUu2sZqhPggLzc6',
             'https://tapwage.com/job/49810371-phd-scientist-internship?context=8mTP4g47UKLKNXZRpi7RGR3QBt1KRtsuNHkHYmHkencdkwLFVTQyerAYtMDz8U4a5XCkZXFwdiueUaB1ZhGNM7JdiMwsrB7hhCuGPwdkNTKZWn3Hd1n7tVAx99MkGSfcGMcTAerjypqivcbixWgYuNUMQXUu2sZqhPggLzc6']
test = get_multiple_jds(test_list)

{'title': 'PhD Program & Lab Manager (Hybrid) job', 'company': 'Mass General Brigham', 'location': 'Charlestown, MA', 'uid': '48022361 ', 'tags': ['PhD', 'Masters Degree', "Bachelor's Degree", 'Entry-Level', 'Junior', 'Non-Profit', 'Healthcare', 'Full-Time', 'Lab Management'], 'description': "DescriptionJob Summary:The School of Health and Rehabilitation Sciences (SHRS) is seeking someone to fill a dual role, working closely with Dr. Teresa J Kimberley, PT, PhD, FAPTA, Director of the PhD program in Rehabilitation Sciences (50%) and Director of the Brain Recovery Lab (50%). The PhD Program Manager will be responsible for general administrative and operations functions including student support and organization, budget, personnel, grant administration, academic proposals, accreditation reports, and policy interpretation in the PhD in Rehabilitation Science program. This position provides administrative support to the Program/Department/Lab Directors and/or their designees. The Program M

{'title': 'PhD Program Administrator (Academic Services Senior Professional) job', 'company': 'University of Colorado', 'location': 'Aurora, CO', 'uid': '48304470 ', 'tags': ['PhD', "Bachelor's Degree", 'Universities', 'Event Mgmt', 'Higher Ed Admin', 'Mid-Level', 'Full-Time'], 'description': 'University StaffDescriptionUniversity of Colorado | CU Anschutz Medical CampusSchool of Medicine, Office of Medical Education PhD Program Administrator (Academic Services Senior Professional) Position #00795306 – Requisition #25321Applications are accepted electronically ONLY at www.cu.edu/cu-careers *The University of Colorado has a requirement for COVID-19 vaccinations and full completion thereof by 9/1/21 or upon start date. Information regarding this requirement, and exemptions can be found at:Anschutz: https://www.ucdenver.edu/docs/librariesprovider284/default-document-library/3000-general-admission/3012---covid-19-vaccination-requirement-and-compliance.pdf?sfvrsn=4e9df3ba_2Denver: https://w

{'title': 'PhD Scientist Internship job', 'company': 'Danaher Corporation', 'location': 'Madison, WI', 'uid': '49810371 ', 'tags': ['PhD', 'Fortune500', 'Publicly Listed', 'Conglomerate', 'Entry-Level', 'Internship', 'Junior', 'Biology', 'Industrials', 'Healthcare'], 'description': "Aldevron is an industry pioneer with a core competency in manufacturing for gene and cell therapy. We supply high-quality plasmid DNA, mRNA and proteins to our clients, who are breaking new ground on the world’s most important and challenging solutions to benefit us all. We're looking for innovative, forward-thinking individuals who share our goals of advancing science and making an impact on the world. Aldevron is proud to work alongside a community of nine fellow Danaher Life Sciences companies. Together, we’re pioneering the future of science and medicine, developing products that enable researchers in the fight to save lives.Job Summary:The PhD Intern assists with complex projects in the areas of recomb

In [60]:
## See if it will go into a dataframe!!
df = pd.DataFrame(test)
df.head()

,title,company,location,uid,tags,description,bullets,headers
0,PhD Program & Lab Manager (Hybrid) job,Mass General Brigham,"Charlestown, MA",48022361,"[PhD, Masters Degree, Bachelor's Degree, Entry...",DescriptionJob Summary:The School of Health an...,"[Maintains appropriate office systems, word pr...","[Job Summary:, Job Duties:, Program Manager Ro..."
1,PhD Program Administrator (Academic Services S...,University of Colorado,"Aurora, CO",48304470,"[PhD, Bachelor's Degree, Universities, Event M...",University StaffDescriptionUniversity of Color...,[Applications are accepted electronically ONLY...,[University of Colorado | CU Anschutz Medical ...
2,PhD Scientist Internship job,Danaher Corporation,"Madison, WI",49810371,"[PhD, Fortune500, Publicly Listed, Conglomerat...",Aldevron is an industry pioneer with a core co...,[Express recombinant proteins in microbial or ...,"[Job Summary:, Responsibilities:, Qualificatio..."


In [180]:
def get_urls_on_one_page(search_url, how_many_pages):
    ## Function takes url for page of search results and returns a list of all the urls for each
    ## individual job description page.
    
    response = requests.get(search_url)
    page_html = response.text
    soup = bs(page_html)
    
    urls = []
    try:
        job_entry = soup.find_all(class_ = 'search-item')
        for item in job_entry:
        #print(item.find('a')['href'])
            urls.append(item.find('a')['href'])
        
    except TypeError:
        print('Skip')
                    
    #print('first print')
    #print(urls)
    #print(len(urls))
    #print()
    time.sleep(2)
    
    next_link = soup.find_all(class_ = 'next')
    for item in next_link:
        #print(item.find('a'))
        if item.find('a')['data-page'] != str(how_many_pages):
            next_page = get_urls_on_one_page(item.find('a')['href'], how_many_pages)
            for link in next_page:
                urls.append(link)
            #urls.append(next_page)
            #print('second print')
            #print(urls)
           # print(len(urls))
           # print()
           # next_url.append(item.find('a')['href'])
           # print(next_url)
        else:
            pass
    print(len(urls))
    return(urls)
    

   

In [ ]:
  next_link = soup.find_all(class_ = 'next')
    for item in next_link:
        #print(item.find('a'))
        if item.find('a')['data-page']:
            next_page = get_urls_on_one_page(item.find('a')['href'])
            urls.append(next_page)
            print('second print')
            print(urls)
            print(len(urls))
            print()
           # next_url.append(item.find('a')['href'])
           # print(next_url)
        else:
            pass
    
    return(urls)

In [130]:
all_urls_list = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states', 50)

10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11


In [161]:
five_test = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states', 5)

10
20
30
40
50


In [171]:
len(five_test)

50

In [172]:
batch_1 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states', 50)


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [173]:
batch_1[-1]

'https://tapwage.com/job/49074611-senior-software-engineer?context=bHKxPSw7Ckbdbfdf8x8Ym6sfPpgiNYp2nRjksgLH5Qv9t63Kn1xSoNRpmFCPFcAFPdr55USdS9tjW5sWgFXYvqdehYe8TFjVnty5gw7ufXTg1ncLggLPhUcYMR13C5hrDUGcmxrkWziHHzpsSoMEFdejejLCBrf7MGTHxDDuz'

In [181]:
batch_2 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=51', 100)


Skip
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
429
439
449
459
469
479
489
499


In [182]:
batch_3 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=101', 150)


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [184]:
batch_3[-1]

'https://tapwage.com/job/48597776-r-d-engineer-advanced-tech-development-4-pke?context=3cJcMsfF7qhJPVqPszSMfRyQX8yokEQa4dXRUCBXcUSd4AyZczB5TxbZYtnTtNvUjmZiY9aXX9DXSgRgxGi8fYhAdYPM9R2bisiKLyFiGq6jPHifKN8hWtLVALR1q26qkTKAASZKpZh9utx65MbuzpDyKZzFmMsZ9W7SgxBdaGQ'

In [185]:
batch_4 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=151', 200)



Skip
Skip
Skip
Skip
10
20
30
40
40
42
42
43
53
63
73
83
93
103
113
123
133
143
153
163
173
183
193
203
213
223
233
243
253
263
273
283
293
303
313
323
333
343
353
363
373
383
393
403
413
423
433
443
453
463


In [186]:
batch_5 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=201', 250)


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [187]:
batch_5[0]

'https://tapwage.com/job/24130921-applied-scientist?context=3cJcMsfF7qhJPVqPszSMfRyQX8yokEQa4dUwGU4G7DWMWfTw4XQB1anCDkyVxcZvuvMJvrYkgU7DpecF88S2rv4jPh73UTHqmcA5bMp9js8Arwr6ohQfr4KEL8BYwG98HRmcThjjcqah7oKqq1Gm9iZCdtV5auzD9kV5ySXaG26'

In [188]:
batch_6 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=251', 300)

    

Skip
Skip
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
195
205
214
224
234
244
254
264
274
284
294
304
314
324
334
344
354
364
374
384
394
404
414
424
434
444
454
464
474
484
494


In [189]:
batch_7 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=301', 400)


Skip
Skip
Skip
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
396
397
407
417
427
433
443
453
463
473
483
493
503
513
523
533
543
553
563
573
583
593
603
613
623
633
643
653
663
673
683
693
703
713
723
733
743
753
763
773
783
793
803
813
823
833
843
853
863
873
883
893
903
913
923
933
943
953
963
973
983


In [190]:
batch_8 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=401', 500)


Skip
Skip
Skip
Skip
Skip
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
585
590
592
601
608
618
628
638
648
658
668
678
688
698
708
718
728
738
748
758
768
778
788
798
808
818
828
838
848
858
868
878
888
898
908
918
928
938
948
958
968
978


In [191]:
batch_9 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=501', 600)


Skip
Skip
Skip
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
743
753
753
763
773
783
789
799
809
819
829
839
849
859
869
879
889
899
909
919
929
939
949
959
969
979


In [192]:
batch_10 = get_urls_on_one_page('https://tapwage.com/search?q=phd&l=united+states&search_key=4a2cf0248bfabcce9fef50aa69bd38821988fc2cc5b8c5d5bc5ff75ed7d2f2cd2697221&page=501', 600)


Skip
Skip
Skip
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
743
753
753
763
773
783
789
799
809
819
829
839
849
859
869
879
889
899
909
919
929
939
949
959
969
979


## Batch 1

In [197]:
## try to get JDs for urls in batch 1
test_1 = get_multiple_jds(batch_1)

IndexError: split4
IndexError: split4
IndexError: split4
IndexError: split4
IndexError: split4
IndexError: split4
IndexError: split4
IndexError: split4
IndexError: split4


In [198]:
## See if it will go into a dataframe!!
df_test_1 = pd.DataFrame(test_1)
df_test_1.head()

,title,company,location,uid,tags,description,bullets,headers
0,PhD Program & Lab Manager (Hybrid) job,Mass General Brigham,"Charlestown, MA",48022361,"[PhD, Masters Degree, Bachelor's Degree, Entry...",DescriptionJob Summary:The School of Health an...,"[Maintains appropriate office systems, word pr...","[Job Summary:, Job Duties:, Program Manager Ro..."
1,PhD Program & Lab Manager (Hybrid) job,Massachusetts General Hospital,"Charlestown, MA",47983977,"[PhD, Masters Degree, Bachelor's Degree, Entry...",DescriptionJob Summary:The School of Health an...,"[Maintains appropriate office systems, word pr...","[Job Summary:, Job Duties:, Program Manager Ro..."
2,Assistant Professor - Clinical PhD Program - L...,Bertelsmann,"Los Angeles, CA",48973936,"[PhD, Fortune500, Media, Junior, Full-Time]",Assistant Professor - Clinical PhD Program - L...,[Active scholarship and/or relevant teaching e...,"[Our Mission:, Our Vision:, Duties & Responsib..."
3,Data Scientist / Statistician Intern (BS / MS ...,Lubrizol,"Hazelwood, DE",39232138,"[PhD, Bachelor's Degree, Data Science, Program...","Lubrizol, a Berkshire Hathaway company, is a m...",[Create predictive models by mining complex da...,[Data Scientist / Statistician Intern (BS/MS/P...
4,PhD Scientist Internship job,Danaher Corporation,"Madison, WI",49810371,"[PhD, Fortune500, Publicly Listed, Conglomerat...",Aldevron is an industry pioneer with a core co...,[Express recombinant proteins in microbial or ...,"[Job Summary:, Responsibilities:, Qualificatio..."


In [200]:
df_test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        500 non-null    object
 1   company      500 non-null    object
 2   location     500 non-null    object
 3   uid          500 non-null    object
 4   tags         500 non-null    object
 5   description  500 non-null    object
 6   bullets      500 non-null    object
 7   headers      500 non-null    object
dtypes: object(8)
memory usage: 31.4+ KB


In [204]:
df_test_1.tail()

,title,company,location,uid,tags,description,bullets,headers
495,"Software Engineer, Object Recognition and Pose...",Magic Leap,"Austin, TX",49704393,"[Startups, Unicorns, Moonshots, PhD, Bachelor'...",PhD and Masters University Grad Position-Our c...,[Improvement of object pose estimation pipelin...,"[PhD and Masters University Grad Position-, Jo..."
496,"US Senior Scientist, Clinical job",Boehringer Ingelheim,"Saint Joseph, MO",49371578,"[Senior, PhD, Masters Degree, Pharma / Biotech...",Job DescriptionDescription:Responsible for the...,[Independently initiates and coordinates strat...,"[Description:, Duties & Responsibilities:, Req..."
497,Clinical Chemist job,Memorial Sloan Kettering,"New York City, NY",49265630,"[PhD, Non-Profit, Universities, Hospitals, Hea...",Job DescriptionThe Department of Pathology and...,[],"[Job Description, Closing]"
498,Unit Manager job,Texas Tech University,"Lubbock, TX",49220296,"[PhD, Bachelor's Degree, Universities, CPA, Ju...",Extended Job TitleUnit ManagerPosition Descrip...,"[Reconcile p-cards monthly, with expenditures ...",[]
499,Senior Software Engineer job,Visa,"Foster City, CA",49074611,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,"[Design, implement and evolve highly scalable ...","[Join Visa: A Network Working for Everyone., R..."


In [205]:
## Pickle this dataframe!
df_test_1.to_pickle('batch_1_jds')

## Batch 2

In [206]:
## try to get JDs for urls in batch 2
test_2 = get_multiple_jds(batch_2)

IndexError: split4
Post-Doctoral Fellow - Aldrovandi Laboratory at UCLA job at University of California Los Angeles in Los Angeles, CA - 36638370 | Tapwage Job Search
IndexError: split4
Assistant Professor - Mathematical Sciences (Data Science) View Details job at University of Texas at Dallas in Richardson, TX - 46394674 | Tapwage Job Search
IndexError: split4
Assistant or Associate Professor, Biomedical Engineering job at University at Buffalo in Buffalo, NY - 50046266 | Tapwage Job Search
IndexError: split4
Clinical Assistant / Associate Professor (12); Assistant / Associate Professor HS job at University at Buffalo in Buffalo, NY - 50046219 | Tapwage Job Search
IndexError: split4
Research Associate View Details job at University of Texas at Dallas in Richardson, TX - 49103097 | Tapwage Job Search
IndexError: split4
Associate Professor, Philosophy job at University at Buffalo in Buffalo, NY - 50046222 | Tapwage Job Search
IndexError: split4
Leah and Paul Lewis Chair of Holocaust Stu

In [207]:
## See if it will go into a dataframe!!
df_test_2 = pd.DataFrame(test_2)
df_test_2.head()

,title,company,location,uid,tags,description,bullets,headers
0,Staff Research Scientist - Identity and Authen...,Visa,"Atlanta, GA",48994948,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[Conduct fundamental and applied research on d...,"[Join Visa: A Network Working for Everyone., R..."
1,PostDoctoral Scholar - Robotic / Force Control...,University of California Berkeley,"June, MO",48792544,"[PhD, Mechanical Engineering, Postdoc, Univers...",Position overviewSalary range: The annual sala...,[Tactile sensor design and integration with TM...,"[Salary range:, Percent time:, Anticipated sta..."
2,"Assistant Professor, Information Design and Da...",Northeastern University,"Boston, MA",47943028,"[PhD, MFA, Bachelor's Degree, Universities, Da...","About Northeastern:Founded in 1898, Northeaste...",[],"[About Northeastern:, About the Opportunity:, ..."
3,Research Scholar - Cancer Research Program job,Eastern Virginia Medical School,"Norfolk, VA",47615956,"[PhD, Universities, Junior, Full-Time, MD]","OverviewCancer Biology Laboratory (PI, Dr. Amy...",[],"[Overview, Responsibilities, Qualifications, L..."
4,Research Fellow / Day / 40 HRS / BWH Neurology...,Brigham and Women's Hospital,"Boston, MA",47550874,"[PhD, Biology, Hospitals, Postdoc, Full-Time]",DescriptionSynaptic and Network Development (S...,[PhD degree (or about to be awarded at PhD) in...,"[Expertise required, PhD degree (or about to b..."


In [208]:
df_test_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        499 non-null    object
 1   company      499 non-null    object
 2   location     499 non-null    object
 3   uid          499 non-null    object
 4   tags         499 non-null    object
 5   description  499 non-null    object
 6   bullets      499 non-null    object
 7   headers      499 non-null    object
dtypes: object(8)
memory usage: 31.3+ KB


In [209]:
## Pickle this dataframe!
df_test_2.to_pickle('batch_2_jds')

## Batch 3

In [210]:
## try to get JDs for urls in batch 3
test_3 = get_multiple_jds(batch_3)

IndexError: split4
Assistant Professor of Teaching job at University at Buffalo in Buffalo, NY - 50046267 | Tapwage Job Search
IndexError: split4
Assistant, Associate or Full Professor job at University at Buffalo in Buffalo, NY - 50046252 | Tapwage Job Search
IndexError: split4
PostDoctoral Fellow job at University of Texas at Austin in Austin, TX - 45902613 | Tapwage Job Search
IndexError: split4
Open - Rank Professor of Instruction - Marketing View Details job at University of Texas at Dallas in Richardson, TX - 50127554 | Tapwage Job Search
IndexError: split4
Assistant / Associate Professor job at University at Buffalo in Buffalo, NY - 50046274 | Tapwage Job Search
IndexError: split4
PostDoctoral Associate job at University at Buffalo in Buffalo, NY - 50046155 | Tapwage Job Search
IndexError: split4
PostDoctoral Associate, Psychology job at University at Buffalo in Buffalo, NY - 50046109 | Tapwage Job Search
IndexError: split4
Adjunct Instructor, Real Estate Development job at Univ

In [211]:
## See if it will go into a dataframe!!
df_test_3 = pd.DataFrame(test_3)
df_test_3.head()

,title,company,location,uid,tags,description,bullets,headers
0,Post Doctoral Fellow - Division of Digestive D...,Emory University,"Atlanta, GA",46138113,"[PhD, Universities, Junior, Full-Time, MD]",Discover Your Career at Emory UniversityEmory ...,[],"[Discover Your Career at Emory University, Des..."
1,Assistant Research Scientist job,Texas A&M University,"College Station, TX",46026714,"[PhD, Universities, Junior, Programming, Data ...",Job TitleAssistant Research ScientistAgencyTex...,"[Appropriate PhD, Relevant professional experi...","[Job Title, Agency, Department, Proposed Minim..."
2,Program Administrator (Academic Services Profe...,University of Colorado,"Aurora, CO",45989129,"[PhD, Bachelor's Degree, Universities, Event M...",University StaffDescriptionUniversity of Color...,[Applications are accepted electronically ONLY...,[University of Colorado | CU Anschutz Medical ...
3,Cross Domain Solutions (CDS) System Engineer job,Peraton,"Annapolis Junction, MD",45950924,"[Senior, PhD, Bachelor's Degree, IT Services, ...",Peraton drives missions of consequence spannin...,"[Bachelors Degree in System Engineering, Compu...","[Basic Qualifications:, Preferred Qualificatio..."
4,Server Performance Architect - 136824 job,AMD,"Austin, TX",45935025,"[PhD, Bachelor's Degree, Fortune500, Publicly ...","What you do at AMD changes everything At AMD, ...","[Impact microarchitecture exploration, perform...","[What you do at AMD changes everything, The Ro..."


In [212]:
df_test_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        500 non-null    object
 1   company      500 non-null    object
 2   location     500 non-null    object
 3   uid          500 non-null    object
 4   tags         500 non-null    object
 5   description  500 non-null    object
 6   bullets      500 non-null    object
 7   headers      500 non-null    object
dtypes: object(8)
memory usage: 31.4+ KB


In [213]:
## Pickle this dataframe!
df_test_3.to_pickle('batch_3_jds')

## Batch 4

In [214]:
## try to get JDs for urls in batch 4
test_4 = get_multiple_jds(batch_4)

IndexError: split4
Open Rank Professor View Details job at University of Texas at Dallas in Richardson, TX - 48090083 | Tapwage Job Search
IndexError: split4
Assistant / Associate / Professor of Instruction - Business Communications View Details job at University of Texas at Dallas in Richardson, TX - 46944841 | Tapwage Job Search


In [215]:
## See if it will go into a dataframe!!
df_test_4 = pd.DataFrame(test_4)
df_test_4.head()

,title,company,location,uid,tags,description,bullets,headers
0,Research Lab Manager 1 - Mendillo Lab (New) job,Northwestern University,"Chicago, IL",48594634,"[PhD, Masters Degree, Bachelor's Degree, Unive...",Department: MED-Biochem&Molecular GeneticsSala...,[Manages technical operations of a lab involve...,"[Department:, Salary/Grade:, Job Summary:, Spe..."
1,Assistant Professor (Posting) job,Northwestern University,"Evanston, IL",48594600,"[PhD, MFA, Masters Degree, Universities, Full-...",Assistant ProfessorPerformance Studies & Afric...,"[Advanced degree (MFA, PhD, DMA, etc.) or equi...","[Assistant Professor, Performance Studies & Af..."
2,"Senior Product Manager, Visa Direct Cross Bord...",Visa,"Miami, FL",48584235,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[End to end deployment of key enablers to supp...,"[Join Visa A Network Working for Everyone., Re..."
3,"Senior Risk Manager, Regulatory & Risk Governa...",Visa,"Atlanta, GA",48583939,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[U.S. supervisory exams and reviews of Visa by...,"[Join Visa: A Network Working for Everyone., E..."
4,Deloitte Risk & Financial Advisory Solution De...,Deloitte,"Denver, CO",38982822,"[Consulting Firms, Auditors, Fortune500, Partn...",Position SummaryAre you interested in improvin...,[Learn to manage and operate one or more indus...,[Work you’ll do]


In [216]:
df_test_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        463 non-null    object
 1   company      463 non-null    object
 2   location     463 non-null    object
 3   uid          463 non-null    object
 4   tags         463 non-null    object
 5   description  463 non-null    object
 6   bullets      463 non-null    object
 7   headers      463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [217]:
## Pickle this dataframe!
df_test_4.to_pickle('batch_4_jds')

## Batch 5

In [218]:
## try to get JDs for urls in batch 5
test_5 = get_multiple_jds(batch_5)

IndexError: split4
PostDoctoral Associate, Geography job at University at Buffalo in Buffalo, NY - 50046285 | Tapwage Job Search
IndexError: split4
PostDoctoral Researcher, Medicine job at University at Buffalo in Buffalo, NY - 50046210 | Tapwage Job Search
IndexError: split4
Clinical Assistant Professor, Japanese Cultural Studies job at University at Buffalo in Buffalo, NY - 50046059 | Tapwage Job Search


In [219]:
## See if it will go into a dataframe!!
df_test_5 = pd.DataFrame(test_5)
df_test_5.head()

,title,company,location,uid,tags,description,bullets,headers
0,Applied Scientist job,Amazon,"Seattle, WA",24130921,"[PhD, Masters Degree, Fortune500, Conglomerate...",DESCRIPTIONDo you want to join an innovative t...,"[, Use statistical and machine learning techni...",[]
1,"Software Engineering Intern, DL Inference Depl...",NVIDIA,"Santa Clara, CA",23554431,"[PhD, Masters Degree, Bachelor's Degree, Entry...",NVIDIA has continuously reinvented itself over...,[Develop production grade GPU-accelerated ML i...,"[What you’ll be doing:, What we need to see:, ..."
2,PostDoctoral Research Scientist job,Columbia University,"New York City, NY",24371884,"[PhD, Universities, Postdoc, Full-Time]",Summary DescriptionThe Division of Molecular G...,[],[]
3,Faculty. Instructor. Research job,The University of Pittsburgh,"Pittsburgh, PA",21061346,"[PhD, Universities, Junior, Programming, Lectu...",Med-Neurological Surgery-Pennsylvania-Pittsbur...,[],"[Research Instructor, Lead Neurophysiologist f..."
4,PostDoctoral Researcher - Computational Flow M...,NREL,"Golden, CO",19834346,"[PhD, Government, Energy, CleanTech, Research ...",Posting Title\nPostdoctoral Researcher – Compu...,[Perform simulations with little to no oversig...,"[Posting Title, Location, Position Type, Hours..."


In [220]:
df_test_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        500 non-null    object
 1   company      500 non-null    object
 2   location     500 non-null    object
 3   uid          500 non-null    object
 4   tags         500 non-null    object
 5   description  500 non-null    object
 6   bullets      500 non-null    object
 7   headers      500 non-null    object
dtypes: object(8)
memory usage: 31.4+ KB


In [221]:
## Pickle this dataframe!
df_test_5.to_pickle('batch_5_jds')

## Batch 6

In [222]:
## try to get JDs for urls in batch 6
test_6 = get_multiple_jds(batch_6)

IndexError: split4
Assistant / Associate / Professor - Speech, Language, Hearing View Details job at University of Texas at Dallas in Richardson, TX - 45197638 | Tapwage Job Search
IndexError: split4
Lecturer - Mechanical Engineering View Details job at University of Texas at Dallas in Richardson, TX - 40708424 | Tapwage Job Search
IndexError: split4
PostDoctoral Research Associate, Physics job at University at Buffalo in Buffalo, NY - 50046215 | Tapwage Job Search
IndexError: split4
Assistant / Associate Professor 10M job at University at Buffalo in Buffalo, NY - 50046152 | Tapwage Job Search
IndexError: split4
Adjunct Instructor, Urban and Regional Planning job at University at Buffalo in Buffalo, NY - 50046012 | Tapwage Job Search


In [223]:
## See if it will go into a dataframe!!
df_test_6 = pd.DataFrame(test_6)
df_test_6.head()

,title,company,location,uid,tags,description,bullets,headers
0,Medical Writer job,Syneos Health,"New York City, NY",47098421,"[Senior, PhD, Pharma / Biotech, Healthcare, Wr...",OverviewCan you reach people with just a few w...,[Acts as agency expert on assigned products an...,"[Overview, Can you reach people with just a fe..."
1,Downstream Laboratory Associate Scientist II job,AstraZeneca,"Frederick, MD",47079815,"[PhD, Bachelor's Degree, Pharma / Biotech, For...","At AstraZeneca, we work together across global...",[This is posted as an Associate Scientist II /...,"[What you’ll do:, Education:, Why AstraZeneca?..."
2,Signal Processing Engineer job,General Atomics,"Boulder, CO",46996084,"[Senior, PhD, Masters Degree, Bachelor's Degre...","General Atomics (GA), and its affiliated compa...","[Function in an advisory or leadership role, p...","[DUTIES AND QUALIFICATIONS:, Job Qualifications:]"
3,PostDoctoral Scholar Research Associate job,University of Southern California,"Los Angeles, CA",46966064,"[PhD, Education, Universities, Postdoc, Academ...",Postdoctoral Scholar – Research Associate Keck...,[Performing genomic and epigenomic experiments...,"[Postdoctoral Scholar – Research Associate, Ke..."
4,TTU Post Doctoral Research Associate job,Texas Tech University,"Lubbock, TX",46926714,"[PhD, Universities, Junior, Full-Time, Academi...",Extended Job TitleTTU Post Doctoral Research A...,"[Cover Letter, Resume/CV, Summary of research ...",[Please provide the following with your online...


In [224]:
df_test_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        494 non-null    object
 1   company      494 non-null    object
 2   location     494 non-null    object
 3   uid          494 non-null    object
 4   tags         494 non-null    object
 5   description  494 non-null    object
 6   bullets      494 non-null    object
 7   headers      494 non-null    object
dtypes: object(8)
memory usage: 31.0+ KB


In [225]:
# Pickle this dataframe!
df_test_6.to_pickle('batch_6_jds')

## Batch 7

In [226]:
## try to get JDs for urls in batch 7
test_7 = get_multiple_jds(batch_7)

IndexError: split4
Assistant / Associate / Professor of Instruction - Psychology View Details job at University of Texas at Dallas in Richardson, TX - 48263636 | Tapwage Job Search
IndexError: split4
Assistant / Associate / Professor of Instruction - Human Development and Early Childhood Disorders View Details job at University of Texas at Dallas in Richardson, TX - 46107928 | Tapwage Job Search
IndexError: split4
PostDoctoral Scholars in The Department of Earth, Planetary, and Space Sciences at The University of California, Los Angeles (Epss / UCLA) job at University of California Los Angeles in Los Angeles, CA - 45292122 | Tapwage Job Search
IndexError: split4
Assistant Professor, Educator, Communications at Clermont College job at University of Cincinnati in Cincinnati, OH - 45069473 | Tapwage Job Search
IndexError: split4
2022 - 23 Professional Track Faculty - Policy Communications job at University of Texas at Austin in Austin, TX - 50158556 | Tapwage Job Search
IndexError: split4

In [227]:
## See if it will go into a dataframe!!
df_test_7 = pd.DataFrame(test_7)
df_test_7.head()

,title,company,location,uid,tags,description,bullets,headers
0,PostDoctoral Research Fellow - Metabolic Disea...,Merck & Co.,"South San Francisco, CA",49581145,"[PhD, Pharma / Biotech, Fortune500, Publicly L...",Job DescriptionNew hires in office-based roles...,[The successful candidate should have a clear ...,"[Job Description, Be a part of the legacy: Pos..."
1,"Manager, Pharmacokinetics, Dynamics and Metabo...",Sanofi,"Cambridge, MA",49566104,"[PhD, Pharma / Biotech, Healthcare, Junior, Pu...","The Pharmacokinetics, Dynamics, and Metabolism...",[Provide input to project strategy under super...,"[Job Summary:, Job Details:, Minimum requireme..."
2,"Senior Development Consultant, Visa Rules Mana...",Visa,"Ashburn, VA",49213631,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[Responsible for ensuring Visa Direct Rules re...,"[Join Visa: A Network Working for Everyone., E..."
3,Senior Employee Relations Manager job,Visa,"Austin, TX",49558503,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[Partner with managers/employees to facilitate...,"[Join Visa: A Network Working for Everyone., E..."
4,"Director, Employee Resource Groups job",Visa,"Foster City, CA",49558496,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[Increase and advance diverse talent through a...,"[Join Visa: A Network Working for Everyone., W..."


In [228]:
df_test_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        983 non-null    object
 1   company      983 non-null    object
 2   location     983 non-null    object
 3   uid          983 non-null    object
 4   tags         983 non-null    object
 5   description  983 non-null    object
 6   bullets      983 non-null    object
 7   headers      983 non-null    object
dtypes: object(8)
memory usage: 61.6+ KB


In [229]:
# Pickle this dataframe!
df_test_7.to_pickle('batch_7_jds')

## Batch 8

In [230]:
## try to get JDs for urls in batch 8
test_8 = get_multiple_jds(batch_8)


IndexError: split4
Breast Tumor Immunology Postdoc in Guerriero Lab at BWH job at Brigham and Women's Hospital in Boston, MA - 47311057 | Tapwage Job Search
IndexError: split4
Post Doctoral Research Associate View Details job at University of Texas at Dallas in Richardson, TX - 47109939 | Tapwage Job Search
IndexError: split4
Marvin and Kathleen Stone Distinguished Professor of Humanities in Medicine and Science View Details job at University of Texas at Dallas in Richardson, TX - 46704672 | Tapwage Job Search
IndexError: split4
Breast Tumor Immunology Postdoc in Guerriero Lab at BWH job at Brigham and Women's Hospital in Boston, MA - 45548740 | Tapwage Job Search
IndexError: split4
Clinician - Scientist Faculty in Laboratory Medicine at Yale job at Yale University in New Haven, CT - 39456201 | Tapwage Job Search
IndexError: split4
PostDoctoral Associate job at University at Buffalo in Buffalo, NY - 50046216 | Tapwage Job Search
IndexError: split4
Assistant Professor HS, Biomedical Inf

In [231]:
## See if it will go into a dataframe!!
df_test_8 = pd.DataFrame(test_8)
df_test_8.head()


,title,company,location,uid,tags,description,bullets,headers
0,"Director, Visa Business Solutions, Risk Busine...",Visa,"Foster City, CA",48227471,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[Developing partnerships with First Line of De...,"[Join Visa: A Network Working for Everyone., T..."
1,Senior Cybersecurity Analyst job,Visa,"Foster City, CA",48227342,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[Must be able to work with Global teams with f...,"[Join Visa: A Network Working for Everyone., W..."
2,HPLC / MS Method Development Senior Scientist job,Eurofins,"Lake Forest, IL",48207528,"[Senior, PhD, Bachelor's Degree, Pharma / Biot...",Company DescriptionEurofins Scientific is an i...,[Support the development & validation of LC-MS...,"[Responsibilities will include, but are not li..."
3,"Solution Management, Manager job",Visa,"Foster City, CA",48136365,"[MBA, PhD, Masters Degree, Bachelor's Degree, ...",Company DescriptionVisa is a world leader in d...,[Lead Visa Digital Product commercial rollouts...,"[Join Visa: A Network Working for Everyone., E..."
4,Faculty Professor job,The University of Pittsburgh,"Pittsburgh, PA",48133877,"[PhD, Universities, Full-Time, MD, Bioinformat...",Med-Medicine-Pennsylvania-Pittsburgh-(Co-Direc...,[],[]


In [232]:
df_test_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        978 non-null    object
 1   company      978 non-null    object
 2   location     978 non-null    object
 3   uid          978 non-null    object
 4   tags         978 non-null    object
 5   description  978 non-null    object
 6   bullets      978 non-null    object
 7   headers      978 non-null    object
dtypes: object(8)
memory usage: 61.2+ KB


In [233]:
# Pickle this dataframe!
df_test_8.to_pickle('batch_8_jds')

## Batch 9

In [234]:
## try to get JDs for urls in batch 9
test_9 = get_multiple_jds(batch_9)

IndexError: split4
PostDoctoral Researcher at Vida Lab for Data Mining and Machine Learning With Applications in Health job at New York University in Brooklyn, NY - 35767354 | Tapwage Job Search
IndexError: split4
PostDoctoral Research Associate View Details job at University of Texas at Dallas in Richardson, TX - 50260836 | Tapwage Job Search
IndexError: split4
Thomas B. Lockwood Chair in Latinx History (Associate or Full Professor) job at University at Buffalo in Buffalo, NY - 50046276 | Tapwage Job Search
IndexError: split4
Adjunct Instructor, School of Engineering and Applied Sciences job at University at Buffalo in Buffalo, NY - 50046256 | Tapwage Job Search
IndexError: split4
Assistant Professor, Dance job at University at Buffalo in Buffalo, NY - 50046235 | Tapwage Job Search
IndexError: split4
Educational Workforce Specialist, Clinical and Translational Science Institute job at University at Buffalo in Buffalo, NY - 50046174 | Tapwage Job Search
IndexError: split4
Clinical Assi

In [235]:
## See if it will go into a dataframe!!
df_test_9 = pd.DataFrame(test_9)
df_test_9.head()


,title,company,location,uid,tags,description,bullets,headers
0,"Assistant Professor, Associate Professor or Pr...",Oregon Health & Science University,"Beaverton, OR",45519755,"[PhD, Universities, Hospitals, Healthcare, Jun...",Department OverviewResearch in the field of ma...,[],"[Department Overview, Function/Duties of Posit..."
1,PostDoctoral Scholar job,Oregon Health & Science University,"Portland, OR",45519722,"[PhD, Universities, Hospitals, Healthcare, Pos...",Function/Duties of PositionThis is a junior le...,"[PhD in related field, Successful candidates a...","[Function/Duties of Position, Required Qualifi..."
2,PostDoctoral Scholar job,Oregon Health & Science University,"Portland, OR",45519279,"[PhD, Universities, Hospitals, Healthcare, Jun...",Function/Duties of PositionWe are seeking a re...,"[Manipulating and producing plasmid DNAs, whic...","[Function/Duties of Position, Required Qualifi..."
3,PostDoctoral Scholar job,Oregon Health & Science University,"Portland, OR",45518604,"[PhD, Universities, Hospitals, Healthcare, Mid...",Department OverviewThe OHSU Knight Cancer Inst...,[We act BOLDLY—Breakthroughs require pushing t...,"[Department Overview, Function/Duties of Posit..."
4,"Associate Director of Diversity, Equity, and I...",Oregon Health & Science University,"Portland, OR",45518074,"[PhD, Universities, Hospitals, Healthcare, Mid...",Department OverviewThe Knight Cancer Institute...,[As a leader in addressing healthcare disparit...,"[0.2 FTE, 1.0 FTE, Position Summary:, Goals an..."


In [236]:
df_test_9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979 entries, 0 to 978
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        979 non-null    object
 1   company      979 non-null    object
 2   location     979 non-null    object
 3   uid          979 non-null    object
 4   tags         979 non-null    object
 5   description  979 non-null    object
 6   bullets      979 non-null    object
 7   headers      979 non-null    object
dtypes: object(8)
memory usage: 61.3+ KB


In [237]:
# Pickle this dataframe!
df_test_9.to_pickle('batch_9_jds')

## Batch 10

In [238]:
## try to get JDs for urls in batch 10
test_10 = get_multiple_jds(batch_10)

IndexError: split4
PostDoctoral Researcher at Vida Lab for Data Mining and Machine Learning With Applications in Health job at New York University in Brooklyn, NY - 35767354 | Tapwage Job Search
IndexError: split4
PostDoctoral Research Associate View Details job at University of Texas at Dallas in Richardson, TX - 50260836 | Tapwage Job Search
IndexError: split4
Thomas B. Lockwood Chair in Latinx History (Associate or Full Professor) job at University at Buffalo in Buffalo, NY - 50046276 | Tapwage Job Search
IndexError: split4
Adjunct Instructor, School of Engineering and Applied Sciences job at University at Buffalo in Buffalo, NY - 50046256 | Tapwage Job Search
IndexError: split4
Assistant Professor, Dance job at University at Buffalo in Buffalo, NY - 50046235 | Tapwage Job Search
IndexError: split4
Educational Workforce Specialist, Clinical and Translational Science Institute job at University at Buffalo in Buffalo, NY - 50046174 | Tapwage Job Search
IndexError: split4
Clinical Assi

In [239]:
## See if it will go into a dataframe!!
df_test_10 = pd.DataFrame(test_10)
df_test_10.head()

,title,company,location,uid,tags,description,bullets,headers
0,"Assistant Professor, Associate Professor or Pr...",Oregon Health & Science University,"Beaverton, OR",45519755,"[PhD, Universities, Hospitals, Healthcare, Jun...",Department OverviewResearch in the field of ma...,[],"[Department Overview, Function/Duties of Posit..."
1,PostDoctoral Scholar job,Oregon Health & Science University,"Portland, OR",45519722,"[PhD, Universities, Hospitals, Healthcare, Pos...",Function/Duties of PositionThis is a junior le...,"[PhD in related field, Successful candidates a...","[Function/Duties of Position, Required Qualifi..."
2,PostDoctoral Scholar job,Oregon Health & Science University,"Portland, OR",45519279,"[PhD, Universities, Hospitals, Healthcare, Jun...",Function/Duties of PositionWe are seeking a re...,"[Manipulating and producing plasmid DNAs, whic...","[Function/Duties of Position, Required Qualifi..."
3,PostDoctoral Scholar job,Oregon Health & Science University,"Portland, OR",45518604,"[PhD, Universities, Hospitals, Healthcare, Mid...",Department OverviewThe OHSU Knight Cancer Inst...,[We act BOLDLY—Breakthroughs require pushing t...,"[Department Overview, Function/Duties of Posit..."
4,"Associate Director of Diversity, Equity, and I...",Oregon Health & Science University,"Portland, OR",45518074,"[PhD, Universities, Hospitals, Healthcare, Mid...",Department OverviewThe Knight Cancer Institute...,[As a leader in addressing healthcare disparit...,"[0.2 FTE, 1.0 FTE, Position Summary:, Goals an..."


In [240]:
df_test_9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979 entries, 0 to 978
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        979 non-null    object
 1   company      979 non-null    object
 2   location     979 non-null    object
 3   uid          979 non-null    object
 4   tags         979 non-null    object
 5   description  979 non-null    object
 6   bullets      979 non-null    object
 7   headers      979 non-null    object
dtypes: object(8)
memory usage: 61.3+ KB


In [242]:
##Pickle this dataframe!
df_test_10.to_pickle('batch_10_jds')